<a href="https://colab.research.google.com/github/FanchonSora/AI-Project-Course/blob/main/Medical_prescription_to_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 1. Cài đặt các thư viện cần thiết và Tesseract OCR
!sudo apt-get install tesseract-ocr -qq
!pip install opencv-python Pillow pytesseract pandas -qq
!pip install torch torchvision transformers easyocr sentence-transformers faiss-cpu -q
!sudo apt-get install tesseract-ocr -qq
!pip install easyocr pytesseract opencv-python pillow transformers sentence-transformers faiss-cpu

In [31]:
!pip show easyocr opencv-python PIL


Name: easyocr
Version: 1.7.2
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 
---
Name: opencv-python
Version: 4.11.0.86
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy
Required-by: dopamine_rl, imgaug


In [11]:
# 2. Mount Google Drive chứa dữ liệu
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# 3. Import các thư viện
import os
import cv2
import pytesseract
import easyocr
import pandas as pd
from PIL import Image, ImageDraw
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

In [19]:
# 4. Khởi tạo các mô-đun AI
# --------------------------------------------------------
# a) Khởi tạo EasyOCR cho tiếng Anh.
#    Nếu bạn muốn nhận diện ngôn ngữ khác (VD: tiếng Việt),
#    có thể thêm mã ngôn ngữ tương ứng: ['en', 'vi']
reader = easyocr.Reader(['en'])

# b) Khởi tạo pipeline cho Named Entity Recognition (NER)
#    Sử dụng model "dslim/bert-base-NER" đã được fine-tune
ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

# c) Khởi tạo mô hình SentenceTransformer
#    để tạo embedding vector cho tên thuốc
sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [23]:
# 4. Định nghĩa đường dẫn tới các thư mục và file dữ liệu
# Thư mục chứa ảnh đơn thuốc
# Thư mục lưu các file OCR text tương ứng với ảnh đơn thuốc
# File CSV chứa metadata của DrugBank (giả sử file có cột: drug_id, drug_name, description, [drug_image_path])
# Định nghĩa đường dẫn tới thư mục chứa ảnh của thuốc (drug_images)
# --------------------------------------------------------
prescription_images_folder = '/content/drive/MyDrive/AI_Project/dataset/prescription_images'
prescription_texts_folder = '/content/drive/MyDrive/AI_Project/dataset/prescription_texts'
drugbank_metadata_path = '/content/drive/MyDrive/AI_Project/dataset/drugbank_metadata.csv'
drug_images_folder = '/content/drive/MyDrive/AI_Project/dataset/drug_images'

In [34]:
def ocr_image(image_path):
    """
    Sử dụng EasyOCR để nhận diện chữ trong ảnh đơn thuốc.
    Đầu tiên mở ảnh bằng Pillow (PIL), chuyển thành numpy array rồi gọi EasyOCR.
    """
    from PIL import Image
    import numpy as np
    try:
        # Mở ảnh và chuyển đổi về RGB
        img_pil = Image.open(image_path).convert('RGB')
        # Chuyển sang numpy array
        img_np = np.array(img_pil)
    except Exception as e:
        print("Lỗi khi mở ảnh với PIL:", e)
        return ""

    # Gọi EasyOCR với numpy array
    result = reader.readtext(img_np, detail=0)
    return " ".join(result)

In [21]:
def extract_drug_names(text):
    """
    Sử dụng pipeline NER (Named Entity Recognition) để tìm
    các thực thể có nhãn MISC (thường là tên riêng,
    có thể bao gồm tên thuốc) trong văn bản OCR.
    """
    entities = ner_pipeline(text)
    # Lọc ra những entity có label = 'MISC'
    drug_names = [entity['word'] for entity in entities if entity['entity_group'] == 'MISC']
    return drug_names

In [22]:
def search_drug_info(drug_name, drugbank_df, index, drug_embeddings):
    """
    Tìm kiếm thuốc gần đúng nhất trong DrugBank sử dụng FAISS & Sentence Transformers.

    - drug_name: Tên thuốc tìm kiếm (chuỗi).
    - drugbank_df: DataFrame chứa metadata thuốc (mã, mô tả, v.v.).
    - index: FAISS Index đã được xây dựng.
    - drug_embeddings: Mảng embeddings của cột 'drug_name' trong drugbank_df.
    """
    # Tạo embedding cho từ khóa tìm kiếm
    query_embedding = sbert_model.encode(drug_name, convert_to_tensor=True).cpu().numpy()

    # Tìm trong FAISS Index (top 1 kết quả)
    D, I = index.search(query_embedding.reshape(1, -1), 1)

    best_match_idx = I[0][0] if I[0][0] != -1 else None
    if best_match_idx is not None:
        return drugbank_df.iloc[best_match_idx]
    else:
        return None

In [24]:
# Đọc file CSV chứa metadata thuốc
drugbank_df = pd.read_csv(drugbank_metadata_path, encoding='cp1252')

In [25]:
# 7. Xây dựng FAISS index cho cột 'drug_name'
# Tạo embeddings cho tất cả tên thuốc trong DataFrame
drug_embeddings = sbert_model.encode(drugbank_df['drug_name'].tolist(), convert_to_tensor=True).cpu().numpy()

# Khởi tạo FAISS Index (sử dụng L2 distance)
d = drug_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(drug_embeddings)

In [ ]:
from IPython.display import display  # Đảm bảo đã import display từ IPython

# 8. Vòng lặp xử lý hàng loạt ảnh đơn thuốc
# --------------------------------------------------------
for filename in os.listdir(prescription_images_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(prescription_images_folder, filename)
        print(f"🚀 Đang xử lý ảnh đơn thuốc: {filename}")

        # Bước 1: OCR sử dụng hàm ocr_image
        text = ocr_image(image_path)
        print(f"📄 Kết quả OCR:\n{text}")

        # Bước 2: Trích xuất tên thuốc bằng mô hình NER
        extracted_drugs = extract_drug_names(text)
        print(f"🔍 Tên thuốc nhận diện: {extracted_drugs}")

        # Bước 3: Tìm kiếm thông tin thuốc trong DrugBank (Vector Search)
        matched_drugs = []
        for drug in extracted_drugs:
            drug_info = search_drug_info(drug, drugbank_df, index, drug_embeddings)
            if drug_info is not None:
                matched_drugs.append(drug_info)

        # Bước 4: Hiển thị kết quả
        if matched_drugs:
            print("✅ **Thông tin thuốc được trích xuất:**")
            for drug in matched_drugs:
                print(f"🔹 **Tên:** {drug['drug_name']}")
                print(f"💊 **Mô tả:** {drug['description']}")
                print(f"🔗 **Mã DrugBank:** {drug['drug_id']}")

                # Hiển thị ảnh thuốc nếu có cột 'drug_image_path'
                if 'drug_image_path' in drug and pd.notna(drug['drug_image_path']):
                    # Loại bỏ ký tự "/" thừa ở đầu nếu có
                    drug_img_relative = drug['drug_image_path'].lstrip("/")
                    drug_img_path = os.path.join(drug_images_folder, drug_img_relative)
                    if os.path.exists(drug_img_path):
                        try:
                            # Mở ảnh bằng Pillow (PIL)
                            drug_img = Image.open(drug_img_path)
                            print(f"🖼️  Ảnh thuốc: {drug_img_path}")
                            display(drug_img)
                        except Exception as e:
                            print(f"🚨 Lỗi khi mở ảnh: {e}")
                    else:
                        print(f"🚨 Không tìm thấy ảnh cho thuốc: {drug['drug_name']}")
        else:
            print("❌ Không tìm thấy thuốc nào khớp.")

print("Hoàn tất xử lý tất cả ảnh đơn thuốc!")


🚀 Đang xử lý ảnh đơn thuốc: 1.jpg
📄 Kết quả OCR:
Dr B;, Who Farmstrcct 12 Kirkvillc tel, 3876 uate Lav 1994 ttriw 0.12 m% theoh ~ M0,7 44 hlt Lm Ms/Mr Pehuiof 30 address; 7v Ye+ Agc:
🔍 Tên thuốc nhận diện: []
❌ Không tìm thấy thuốc nào khớp.
🚀 Đang xử lý ảnh đơn thuốc: 10.jpg
📄 Kết quả OCR:
Dr; MRCP (Uk) FrCA etuese Corsultant '{Oantro! CCT (Gasto) Livat Itanuplant #euems"b Apollo [GastRoenteroLoGy No: 46200 Gastroenterolbgisl] and Wransplanl Hepaloloqist 040 2360 77 / Fot Extn 4005i4 Emaappoontments CJi Dem0005/0 A 6 Am : 7102778809 aocpolav@gmail com yle/u To So Eve e IT MAY ConCeRn Thu U Jnlonm ~kat Mr- CK . SAmuel 6lyr | Malz Uu ccuuntly  admuttaf urdu Cau wiJh  Tp.No: 36+s88 ny He 1 (usuty  Wrdqeiy  taliis w CCU sitsaSspsu kr dh Mobs Lui akstex? sNh Axpiraben   pruumonia )  Y6 fuku  needs  lerpilal alout & -l0 768 lomblut TF Jo 6 and Kurdly do -ba hneq ckul &n kprm FFuuki; Yau DE NAVEEN POLAVABAPU RCP Ln) WRCp Eio) VacP C_vo] CCS] (Gazuoi Loas Cansuteni Trcplznl| Rd %o-2255 Hospil

# Mục mới